# Hifi-gan

In [12]:
!pip3 install -U sagemaker
!pip3 install -U boto3

#then restart kernel


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install sagemaker-studio-image-build


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [4]:
!sm-docker build . 

....[Container] 2022/11/09 06:22:47 going inside waitForAgent

[Container] 2022/11/09 06:22:47 Waiting for agent ping
[Container] 2022/11/09 06:22:48 Waiting for DOWNLOAD_SOURCE
[Container] 2022/11/09 06:22:52 Phase is DOWNLOAD_SOURCE
[Container] 2022/11/09 06:22:52 finished waitForAgent
[Container] 2022/11/09 06:22:52 inside CopySrc
[Container] 2022/11/09 06:22:52 CODEBUILD_SRC_DIR=/codebuild/output/src044289546/src
[Container] 2022/11/09 06:22:52 finished CopySrc
[Container] 2022/11/09 06:22:52 YAML location is /codebuild/output/src044289546/src/buildspec.yml
[Container] 2022/11/09 06:22:52 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2022/11/09 06:22:52 Processing environment variables
[Container] 2022/11/09 06:22:52 No runtime version selected in buildspec.
[Container] 2022/11/09 06:22:52 Moving to directory /codebuild/output/src044289546/src
[Container] 2022/11/09 06:22:52 Configuring ssm agent with target id: codebuild:92f931c1-a627-44d4-bbd3-aad78423c6

In [5]:
image_url = "432088571089.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio-d-el61mng51h6v:default-1638805766016"

In [6]:
import os
import json

import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
import torch

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()
prefix = "hifi-gan"
output_path = f"s3://{bucket}/{prefix}"

print("torch.__version__:{}".format(torch.__version__))
print("boto3.__version__:{}".format(boto3.__version__))
print("sagemaker.__version__:{}".format(sagemaker.__version__))
#print("bucket:{}".format(bucket))
print("role:{}".format(role))

torch.__version__:1.12.1+cpu
boto3.__version__:1.26.5
sagemaker.__version__:2.116.0
role:arn:aws:iam::432088571089:role/AmazonSageMaker-ExecutionRole-20210324T123126


In [7]:
#!pygmentize 'train.py'

In [8]:
instance_type = "ml.p3.2xlarge"
#instance_type = "ml.p3.2xlarge" #for testing account
instance_count = 1
checkpoint_s3_uri = f"s3://{bucket}/{prefix}/checkpoints"
#checkpoint_local_path="cp_hifigan"
hyperparameters={"config": "config_v1.json"}

est = PyTorch(
    entry_point="train.py",
    image_uri = image_url,
    source_dir="./code",  # directory of your training script
    role=role,
    framework_version="1.11.0",
    py_version="py38",
    instance_type=instance_type,
    instance_count=instance_count,
    volume_size=200,
    output_path=output_path,
    hyperparameters=hyperparameters,
    disable_profiler=True,
    debugger_hook_config=False,
    base_job_name=prefix,
    checkpoint_s3_uri = checkpoint_s3_uri,
    #checkpoint_local_path=checkpoint_local_path,
)

print(est.training_image_uri())

432088571089.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio-d-el61mng51h6v:default-1638805766016


In [9]:
from sagemaker.inputs import TrainingInput
prefix_dataset = "datasets/LJSpeech1.1_2.6G/wavs"
loc =f"s3://{bucket}/{prefix_dataset}"

training = TrainingInput(
    s3_data_type='S3Prefix', # Available Options: S3Prefix | ManifestFile | AugmentedManifestFile
    s3_data=loc,
    distribution='FullyReplicated', # Available Options: FullyReplicated | ShardedByS3Key 
    input_mode='FastFile'
)

In [11]:
#est.fit(inputs=channels)
#est.fit({"traink mn ing":training,"testing":testing,"test_instance_group":test_instance_group})

job_name = est.fit({"training":training})

2022-11-09 06:51:13 Starting - Starting the training job......
2022-11-09 06:51:51 Starting - Preparing the instances for training......
2022-11-09 06:53:13 Downloading - Downloading input data...
2022-11-09 06:53:36 Training - Downloading the training image...................................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-11-09 06:59:21,330 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-11-09 06:59:21,359 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-11-09 06:59:21,367 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-11-09 06:59:22,008 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 55.0 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
pt_mnist_model_data = est.model_data
print("Model artifact saved at:\n", pt_mnist_model_data)

Model artifact saved at:
 s3://sagemaker-us-east-1-432088571089/hifi-gan/hifi-gan-2022-11-09-06-51-12-719/output/model.tar.gz
